In [107]:
import numpy as np
import pandas as pd
import re
import warnings
import unidecode
from keras.layers import Input,Embedding,Dense,LSTM,TimeDistributed,Concatenate
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings('ignore')

In [108]:
def read_text(filename):
    file=open(filename,mode='rt',encoding='utf-8')
    text=file.read()
    file.close()
    return text

In [109]:
def to_lines(text):
    text=text.split('\n')
    text=[i.split('\t') for i in text]
    return text

In [110]:
text=read_text('spa-eng/spa.txt')
spa_eng=to_lines(text)

In [111]:
spa_eng

[['Go.',
  'Ve.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986655 (cueyayotl)'],
 ['Go.',
  'Vete.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986656 (cueyayotl)'],
 ['Go.',
  'Vaya.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #4986657 (cueyayotl)'],
 ['Go.',
  'Váyase.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #6586271 (arh)'],
 ['Hi.',
  'Hola.',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #431975 (Leono)'],
 ['Run!',
  '¡Corre!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #1685404 (Elenitigormiti)'],
 ['Run!',
  '¡Corran!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #5213896 (cueyayotl)'],
 ['Run!',
  '¡Corra!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #8005613 (Seael)'],
 ['Run!',
  '¡Corred!',
  'CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #8005615 (Seael)'],
 ['Run.',
  'Corre

In [112]:
for i in range(len(spa_eng)):
    spa_eng[i]=spa_eng[i][:2]

In [113]:
spa_eng

[['Go.', 'Ve.'],
 ['Go.', 'Vete.'],
 ['Go.', 'Vaya.'],
 ['Go.', 'Váyase.'],
 ['Hi.', 'Hola.'],
 ['Run!', '¡Corre!'],
 ['Run!', '¡Corran!'],
 ['Run!', '¡Corra!'],
 ['Run!', '¡Corred!'],
 ['Run.', 'Corred.'],
 ['Who?', '¿Quién?'],
 ['Wow!', '¡Órale!'],
 ['Fire!', '¡Fuego!'],
 ['Fire!', '¡Incendio!'],
 ['Fire!', '¡Disparad!'],
 ['Help!', '¡Ayuda!'],
 ['Help!', '¡Socorro! ¡Auxilio!'],
 ['Help!', '¡Auxilio!'],
 ['Jump!', '¡Salta!'],
 ['Jump.', 'Salte.'],
 ['Stop!', '¡Parad!'],
 ['Stop!', '¡Para!'],
 ['Stop!', '¡Pare!'],
 ['Wait!', '¡Espera!'],
 ['Wait.', 'Esperen.'],
 ['Go on.', 'Continúa.'],
 ['Go on.', 'Continúe.'],
 ['Hello!', 'Hola.'],
 ['Hurry!', '¡Date prisa!'],
 ['Hurry!', '¡Daos prisa!'],
 ['Hurry!', 'Dese prisa.'],
 ['I hid.', 'Me oculté.'],
 ['I hid.', 'Me escondí.'],
 ['I hid.', 'Me ocultaba.'],
 ['I hid.', 'Me escondía.'],
 ['I ran.', 'Corrí.'],
 ['I ran.', 'Corría.'],
 ['I try.', 'Lo intento.'],
 ['I won!', '¡He ganado!'],
 ['Oh no!', '¡Oh, no!'],
 ['Relax.', 'Tomátelo con soda

In [114]:
print(len(spa_eng))

125447


In [126]:
spa_eng=np.asarray(spa_eng)
eng_l=[]
spa_l=[]
for i in range(len(spa_eng)-1):
    eng_l.append(spa_eng[i][0])
    spa_l.append(spa_eng[i][1:][0])  

In [127]:
data=pd.DataFrame({'eng':eng_l,'spa':spa_l})

In [128]:
data.tail()

eng  \
125441  There are four main causes of alcohol-related ...   
125442  There are mothers and fathers who will lie awa...   
125443  A carbon footprint is the amount of carbon dio...   
125444  Since there are usually multiple websites on a...   
125445  If you want to sound like a native speaker, yo...   

                                                      spa  
125441  Hay cuatro causas principales de muertes relac...  
125442  Hay madres y padres que se quedan despiertos d...  
125443  Una huella de carbono es la cantidad de contam...  
125444  Como suele haber varias páginas web sobre cual...  
125445  Si quieres sonar como un hablante nativo, debe...

In [129]:
data.drop_duplicates(subset=['spa'],inplace=True)

In [130]:
len(data)

119035

In [134]:
# function to preprocess the text
def cleaner(text):
    newString = text.lower()
    unaccented_string = unidecode.unidecode(newString)
    newString = re.sub("'",'', unaccented_string) 
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = newString.split()
    return (" ".join(tokens)).strip()

In [135]:
data['cleaned_eng']=data['eng'].apply(cleaner)
data['cleaned_spa']=data['spa'].apply(cleaner)

In [136]:
data.tail()

eng  \
125441  There are four main causes of alcohol-related ...   
125442  There are mothers and fathers who will lie awa...   
125443  A carbon footprint is the amount of carbon dio...   
125444  Since there are usually multiple websites on a...   
125445  If you want to sound like a native speaker, yo...   

                                                      spa  \
125441  Hay cuatro causas principales de muertes relac...   
125442  Hay madres y padres que se quedan despiertos d...   
125443  Una huella de carbono es la cantidad de contam...   
125444  Como suele haber varias páginas web sobre cual...   
125445  Si quieres sonar como un hablante nativo, debe...   

                                              cleaned_eng  \
125441  there are four main causes of alcohol related ...   
125442  there are mothers and fathers who will lie awa...   
125443  a carbon footprint is the amount of carbon dio...   
125444  since there are usually multiple websites on a...   
125445  if you want to sound like a native speaker you...   

                                              cleaned_spa  
125441  hay cuatro causas principales de muertes relac...  
125442  hay madres y padres que se quedan despiertos d...  
125443  una huella de carbono es la cantidad de contam...  
125444  como suele haber varias paginas web sobre cual...  
125445  si quieres sonar como un hablante nativo debes...

In [137]:
# train-test split
from sklearn.model_selection import train_test_split
x_tr, x_val, y_tr, y_val = train_test_split(data['cleaned_spa'],data['cleaned_eng'], test_size = 0.2, random_state=0,shuffle=True)

In [138]:
# storing validation data into new variables for later use
x_val_original = x_val
y_val_original = y_val

# reset index
y_val_original.reset_index(inplace=True, drop=True)
x_val_original.reset_index(inplace=True, drop=True)

In [139]:
def build_vocab(texts):
    sentences = texts.apply(lambda x: x.split()).values
    vocab = {}
    for sentence in sentences:
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

# create word-frequency pair dictionary
source_word_freq = build_vocab(x_tr)

In [ ]:
# set threshold value for rare words
thresh=2

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in source_word_freq.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
    freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)